<a href="https://colab.research.google.com/github/gmrammohan15/S4/blob/master/Copy_of_EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # input = 1 , output = 16, kernel size = 3*3
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) # output size remains same 
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1) # output size remains same
        self.pool2D = nn.MaxPool2d(2, 2)  # max pooling kernel =2 , stride =2 

        #1x1 kernel to reduce channel size
        self.conv3 = nn.Conv2d(32, 16, 1)
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1)
  
        self.conv5 = nn.Conv2d(32, 16, 1)
        self.conv6 = nn.Conv2d(16, 32, 3)
        self.conv7 = nn.Conv2d(32, 10, 3)
        self.relu =  nn.ReLU()
        self.BatchNorm16 = nn.BatchNorm2d(16)
        self.BatchNorm32 = nn.BatchNorm2d(32)
        self.dropOut = nn.Dropout2d(0.2)
        self.avgpool2 = nn.AvgPool2d(kernel_size=2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)      
        x = self.BatchNorm16(x)
        x = self.conv2(x)
      
        x = self.relu(x) 
        x = self.dropOut(x)     
        x = self.BatchNorm32(x)
        x = self.pool2D(x)

        x = self.conv3(x)

        x = self.conv4(x)
        x = self.relu(x)
        x = self.dropOut(x)
        x = self.BatchNorm32(x)
        
        x = self.pool2D(x)

        x = self.conv5(x)
   
        x = self.conv6(x)
        x = self.relu(x) 
     
        x = self.BatchNorm32(x)
        x = self.conv7(x)

        x = self.avgpool2(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

In [14]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
              ReLU-2           [-1, 16, 28, 28]               0
       BatchNorm2d-3           [-1, 16, 28, 28]              32
            Conv2d-4           [-1, 32, 28, 28]           4,640
              ReLU-5           [-1, 32, 28, 28]               0
         Dropout2d-6           [-1, 32, 28, 28]               0
       BatchNorm2d-7           [-1, 32, 28, 28]              64
         MaxPool2d-8           [-1, 32, 14, 14]               0
            Conv2d-9           [-1, 16, 14, 14]             528
           Conv2d-10           [-1, 32, 14, 14]           4,640
             ReLU-11           [-1, 32, 14, 14]               0
        Dropout2d-12           [-1, 32, 14, 14]               0
      BatchNorm2d-13           [-1, 32, 14, 14]              64
        MaxPool2d-14             [-1, 3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
#Changed the batch size
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [17]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# 19 epochs
for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    print('\n EPOCH: ' + str(epoch) )

  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0845, Accuracy: 9747/10000 (97%)


 EPOCH: 1


  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0480, Accuracy: 9871/10000 (99%)


 EPOCH: 2


  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0355, Accuracy: 9887/10000 (99%)


 EPOCH: 17


  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0351, Accuracy: 9897/10000 (99%)


 EPOCH: 18


loss=0.015016496181488037 batch_id=937: 100%|██████████| 938/938 [00:15<00:00, 62.04it/s]



Test set: Average loss: 0.0371, Accuracy: 9889/10000 (99%)


 EPOCH: 19
